In [ ]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""

In [24]:
# built-in imports
from time import sleep
import re
import json

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
# from urllib.request import urlopen

import requests
from random import random

!pip install tqdm
from tqdm import tqdm

import csv
import os

import pandas as pd

In [2]:
property_files = '../data/raw/property_meta'
if not os.path.exists(property_files):
    os.makedirs(property_files)

In [8]:
scraped_url = []
if os.path.exists(f'{property_files}/property_url.csv'):
    with open(f'{property_files}/property_url.csv', newline='') as inputfile:
        for row in csv.reader(inputfile):
            scraped_url.append(row[0])
# print(scraped_url)

['https://www.domain.com.au/9-godfrey-avenue-st-kilda-east-vic-3183-16076890', 'https://www.domain.com.au/5-1-3-rex-avenue-alphington-vic-3078-15873554', 'https://www.domain.com.au/805-81-queens-road-melbourne-vic-3004-16067649', 'https://www.domain.com.au/12-1-rex-avenue-alphington-vic-3078-16077577', 'https://www.domain.com.au/3-lime-ct-bellfield-vic-3081-13422950', 'https://www.domain.com.au/24-36-ridley-street-albion-vic-3020-16052264', 'https://www.domain.com.au/586-upper-heidelberg-road-heidelberg-vic-3084-16076963', 'https://www.domain.com.au/21-313-beaconsfield-parade-st-kilda-west-vic-3182-16056043', 'https://www.domain.com.au/1-22-maivary-lane-northcote-vic-3070-16076163', 'https://www.domain.com.au/5-24-passfield-st-brunswick-vic-3056-14597538', 'https://www.domain.com.au/12-45-marine-parade-st-kilda-vic-3182-16059741', 'https://www.domain.com.au/41-chadstone-road-malvern-east-vic-3145-16074988', 'https://www.domain.com.au/9-earl-street-carlton-vic-3053-13480636', 'https://w

In [9]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppelWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# constants
BASE_URL = "https://www.domain.com.au"
sort_methods = ["default-desc", "dateupdated-desc", "price-asc", "price-desc", "suburb-asc"]
N_PAGES = range(1, 51) # update this to your liking

In [10]:
url_links = []
property_metadata = defaultdict(dict)

In [11]:
for sort_method in tqdm(sort_methods):
    for page in tqdm(N_PAGES):
        url = BASE_URL + f"/rent/melbourne-region-vic/?sort={sort_method}&page={page}"
        bs_object = BeautifulSoup(requests.get(url, headers = headers).text, "html.parser")
        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object.find("ul",{"data-testid": "results"}).findAll("a",href=re.compile(f"{BASE_URL}/*"))
        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])
        # sleep(round(random(),2))

100%|██████████| 50/50 [00:36<00:00,  1.37it/s]


In [13]:
print(url_links)

['https://www.domain.com.au/4706-500-elizabeth-street-melbourne-vic-3000-16033929?topspot=1', 'https://www.domain.com.au/5-34-bute-street-murrumbeena-vic-3163-16085475', 'https://www.domain.com.au/7-23-bendigo-avenue-elwood-vic-3184-16085426', 'https://www.domain.com.au/1-12-omar-street-caulfield-south-vic-3162-16085399', 'https://www.domain.com.au/602-250-city-road-southbank-vic-3006-16085393', 'https://www.domain.com.au/26-william-street-dromana-vic-3936-16085361', 'https://www.domain.com.au/2-14-binns-street-montmorency-vic-3094-16085351', 'https://www.domain.com.au/603-101-river-street-south-yarra-vic-3141-16085330', 'https://www.domain.com.au/80a-waverley-street-moonee-ponds-vic-3039-16085329', 'https://www.domain.com.au/12-174-george-street-east-melbourne-vic-3002-14217723', 'https://www.domain.com.au/1-15-tintern-avenue-toorak-vic-3142-16085299', 'https://www.domain.com.au/210-26-warleigh-grove-brighton-vic-3186-15226300', 'https://www.domain.com.au/205-95-ormond-road-elwood-vic

In [17]:
len(url_links)

5001

In [18]:
temp = []
for i in url_links:
    if not i in scraped_url:
        temp.append(i)
url_links = temp

In [19]:
len(url_links)

1687

In [20]:
url_links = list(set(url_links))
num_url = len(url_links)

In [21]:
# urls are stored in a csv file
# as property information are updated real time and scraped over a week
# future scraping should be compared to this to avoid duplication
with open(f'{property_files}/property_url.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    for x in url_links + scraped_url:
        csvwriter.writerow([x])

In [22]:
# for each url, scrape some basic metadata
# this segment of code may need to be modified if domain.com.au makes any changes
# this code is valid at the time of 12 September
for property_url in tqdm(url_links):
    # count += 1
    # bs_object = BeautifulSoup(urlopen(url), "lxml")
    bs_object = BeautifulSoup(requests.get(property_url, headers = headers).text, "html.parser")
    # looks for the header class to get property name
    property_metadata[property_url]['name'] = bs_object.find("h1", {"class": "css-164r41r"}).text
    # print(property_metadata[property_url]['name'])
    property_metadata[property_url]['type'] = bs_object.find("div", {"data-testid": "listing-summary-property-type"}).text
    # looks for the div containing a summary title for cost
    property_metadata[property_url]['cost_text'] = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text
    # print(property_metadata[property_url]['cost_text'])
    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]['coordinates'] = [
        float(coord) for coord in re.findall(
            r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
            bs_object
                .find(
                "a",
                {"target": "_blank", 'rel': "noopener noreferer"}
            )
                .attrs['href']
        )[0].split(',')
    ]
    # print(property_metadata[property_url]['coordinates'])
    property_metadata[property_url]['rooms'] = [
        re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object
            .find("div", {"data-testid": "property-features"})
            .findAll("span", {"data-testid": "property-features-text-container"})
    ]
    # print(property_metadata[property_url]['rooms'])
    property_metadata[property_url]['desc'] = re.sub(r'<br\/>', '\n', str(bs_object.find("p"))).strip('</p>')
    # print(property_metadata[property_url]['desc'])
    # sleep(round(3*random(),2))

100%|██████████| 1324/1324 [13:29<00:00,  1.64it/s]


Save the results of scraping

In [41]:
if os.path.exists(f'{property_files}/property_metadata.json'):
    with open(f'{property_files}/property_metadata.json', 'r') as f:
        data = json.load(f)
    d = defaultdict(list, data)
    for key, value in d.items():
        for subkey, subvalue in value.items():
            property_metadata[key][subkey] = subvalue

In [63]:
with open(f'{property_files}/property_metadata.json', 'w') as f:
    json.dump(property_metadata, f)

In [64]:
if os.path.exists(f'{property_files}/property_metadata.csv'):
    df = pd.read_csv(f'{property_files}/property_metadata.csv')

,url,name,type,cost_text,coordinates,rooms,desc
0,https://www.domain.com.au/5-1-3-rex-avenue-alp...,5/1-3 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$440.00,"[-37.7856068, 145.0249141]","[['2 Beds'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
1,https://www.domain.com.au/805-81-queens-road-m...,805/81 Queens Road Melbourne VIC 3004,Studio,$380 weekly,"[-37.853804, 144.9798785]","[['0 Beds'], ['1 Bath'], ['1 Parking']]",Leasing Concierge
2,https://www.domain.com.au/12-1-rex-avenue-alph...,12/1 Rex Avenue Alphington VIC 3078,Apartment / Unit / Flat,$460.00,"[-37.7856092, 145.0249185]","[['2 Beds'], ['1 Bath'], ['1 Parking']]",Pyper Byrne
3,https://www.domain.com.au/3-lime-ct-bellfield-...,3 Lime Ct Bellfield VIC 3081,House,$425 PW | Weston Real Estate,"[-37.7548292, 145.041952]","[['3 Beds'], ['1 Bath'], ['3 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."
4,https://www.domain.com.au/24-36-ridley-street-...,24/36 Ridley Street Albion VIC 3020,Apartment / Unit / Flat,$250,"[-37.7834829, 144.8233623]","[['1 Bed'], ['1 Bath'], ['1 Parking']]","class=""css-dxogle"">* Unverified feature<svg a..."


In [65]:
df2 = pd.DataFrame(property_metadata).T.reset_index()
df2 = df2.rename(columns = {'index':'url'})

In [66]:
if os.path.exists(f'{property_files}/property_metadata.csv'):
    save_csv = pd.concat([df, df2])
else:
    save_csv = df2

In [62]:
save_csv.to_csv(f'{property_files}/property_metadata.csv', index = False)